In [1]:
import os
import re
import email
import base64

from imaplib import IMAP4_SSL

import config

In [2]:
def message_analizer(message):
    path_saving = '/home/andr/Yandex.Disk/2.Преподавание/НИУ ВШЭ/2019-2020/checking_tasks/test_group/test_student'
    status, data = connection.fetch(message, '(RFC822)')
    msg = email.message_from_bytes(data[0][1], _class = email.message.EmailMessage)
    subject = email.header.decode_header(msg['Subject'])[0][0]
    sender = msg['From'].split()[1][1:-1]
    # блок, по мейлу определяющий отправителя
    time_rec = msg['Date']
    try:
        if b'IMPORTANT' in subject:
            return 'Privat message'
    except TypeError: # Учет того, что иногда в мсг содержится строка, а иногда байты
         if 'IMPORTANT' in subject:
            return 'Privat message from {sender}'
    
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            filename = part.get_filename()
            # Нам плохого не надо, в письме может быть всякое барахло
            if filename:
                if re.match(r'^task_[0-9]+_[0-9]+.py$', filename):
                    with open(path_saving + os.sep + part.get_filename(), 'wb') as new_file:
                        new_file.write(part.get_payload(decode=True))
                else:
                    return f"File {filename} isn't match to the pattern" 
    else:
        return 'No attachemts'
    
    return 'All file were saved'

In [3]:
YA_HOST = "imap.yandex.ru"
YA_PORT = 993
YA_USER = config.log
YA_PASSWORD = config.ps

connection = IMAP4_SSL(host=YA_HOST, port=YA_PORT)
status, content = connection.login(user=YA_USER, password=YA_PASSWORD)
status, content = connection.select('hes_clasroon_tasks', )
status, content = connection.search(None, 'UNSEEN')
unseen_messages = content[0].split()

for message in unseen_messages:
    print(message_analizer(message))

        

error: command SEARCH illegal in state AUTH, only allowed in states SELECTED

In [ ]:
#Определение отправителя исходя из базы (когда натринеруемся на тестах)
#Проверка на то, сорержит ли тема письма импортант
#Проверка на то, приложены ли к письму файлы
#Проверка на то, соответствуют ли файлы шаблону

status, data = connection.fetch(b'5', '(RFC822)')